# Managing Kleio files

> First time use: follow instructions in the [README.md](README.md) file in this directory.

## Initial setup

Create a TimelinkNotebook object. This will be the interface for most of your interactions with Timelink.

The first time you run this notebook, you will be prompted to install ìpykernel. This is required to run Timelink in a notebook.

Note:
* First time run takes a little time as the required Docker images are downloaded.
* Timelink will default to using sqlite as the database, see [Receipts notebook for more control](1-receipts.ipynb)


In [1]:
from timelink.notebooks import TimelinkNotebook

tlnb = TimelinkNotebook()
tlnb.print_info()


Timelink version: 1.1.25
Project name: dehergne
Project home: /Users/jrc/mhk-home/sources/dehergne
Database type: sqlite
Database name: dehergne
Kleio image: timelinkserver/kleio-server
Kleio server token: fyKCz...
Kleio server URL: http://127.0.0.1:8094
Kleio server home: /Users/jrc/mhk-home/sources/dehergne
Kleio server container: interesting_proskuriakova
Kleio version requested: latest
Kleio server version: 12.8.593 (2025-03-16 21:55:53)
SQLite directory: /Users/jrc/mhk-home/sources/dehergne/database/sqlite
Database version: 6ccf1ef385a6
Call print_info(show_token=True) to show the Kleio Server token
Call print_info(show_password=True) to show the Postgres password
TimelinkNotebook(project_name=dehergne, project_home=/Users/jrc/mhk-home/sources/dehergne, db_type=sqlite, db_name=dehergne, kleio_image=timelinkserver/kleio-server, kleio_version=latest, postgres_image=postgres, postgres_version=latest)


TimelinkNotebook created a database to store data and a Kleio server to 
translate transcriptions in Kleio notation into data that can be imported
into a database.

To get information about the database and the Kleio server do ```tnlb.print_info()```

### Database status

Count the number of rows in each table in the database.


In [2]:
tlnb.table_row_count_df()

,table,count
0,acts,29
1,alembic_version,1
2,aregisters,1
3,attributes,26744
4,blinks,200
5,class_attributes,70
6,classes,14
7,entities,32952
8,geoentities,359
9,goods,0


### Show the kleio files available 

In [3]:
kleio_files = tlnb.get_kleio_files()
kleio_files[['name','status','errors','warnings']]

,name,status,errors,warnings
0,dehergne-0-abrev.cli,V,0,0
1,dehergne-a.cli,V,0,0
2,dehergne-b.cli,V,0,0
3,dehergne-c.cli,V,0,0
4,dehergne-d.cli,V,0,0
5,dehergne-e.cli,V,0,0
6,dehergne-f.cli,V,0,0
7,dehergne-g.cli,V,0,0
8,dehergne-h.cli,V,0,0
9,dehergne-i.cli,V,0,0


## Optionally clean translations 

In [4]:
# NBVAL_IGNORE_OUTPUT

# tlnb.kleio_server.translation_clean("", recurse="yes")


# Update the database from Kleio sources.

Updates source translations and imports into database sources with no errors.

Only changed files since last import, or new files, will be processed.

In [9]:
import logging
import os
import time
logging.basicConfig(level=logging.INFO)
# show logging in the notebook
from IPython.core.display import display, HTML
display(HTML("<style>.jp-OutputArea-output { font-size: 1.5em; }</style>"))
# Set up logging to display in the notebook
logger = logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler()
handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
# Set up the logging configuration
# Set the logging level to INFO
# Set the format for the log messages
kleio_home_path = tlnb.kleio_server.get_kleio_home()

# Monitor the directory for changes
def monitor_kleio_home(path):
    logging.info(f"Monitoring changes in {path}")
    previous_files = {f: os.path.getmtime(os.path.join(path, f)) for f in os.listdir(path)}
    while True:
        time.sleep(5)  # Check every 5 seconds
        current_files = {f: os.path.getmtime(os.path.join(path, f)) for f in os.listdir(path)}
        changed_files = [f for f in current_files if f not in previous_files or current_files[f] != previous_files[f]]
        if changed_files:
            logging.info(f"Detected changes in files: {changed_files}")
            tlnb.update_from_sources(path='')
        previous_files = current_files

monitor_kleio_home(kleio_home_path)

/var/folders/x7/qzytnd752559g0w475rfwkx00000gn/T/ipykernel_88880/2590802520.py:6: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


INFO:root:Monitoring changes in /Users/jrc/mhk-home/sources/dehergne
2025-04-13 23:03:48,376 - INFO - Monitoring changes in /Users/jrc/mhk-home/sources/dehergne
INFO:root:Detected changes in files: ['sources']
2025-04-13 23:06:38,536 - INFO - Detected changes in files: ['sources']
INFO:root:Importing sources/dehergne-g.cli
2025-04-13 23:06:45,298 - INFO - Importing sources/dehergne-g.cli
INFO:root:Storing 31 postponed relations
2025-04-13 23:06:52,302 - INFO - Storing 31 postponed relations
INFO:root:Detected changes in files: ['.git']
2025-04-13 23:07:43,437 - INFO - Detected changes in files: ['.git']
INFO:root:Detected changes in files: ['sources']
2025-04-13 23:08:00,397 - INFO - Detected changes in files: ['sources']
INFO:root:Importing sources/dehergne-g.cli
2025-04-13 23:08:07,073 - INFO - Importing sources/dehergne-g.cli
INFO:root:Storing 31 postponed relations
2025-04-13 23:08:13,623 - INFO - Storing 31 postponed relations


KeyboardInterrupt: 

## Check the status of the files

Check the import status of the translated files

* I: Imported
* E: Imported with error
* W: Imported with warnings no errors
* N: Not imported
* U: Translation updated need to reimport

### Check a translation report

If needed to check for errors.


In [44]:

# get the import status
imported_files_df = tlnb.get_import_status()
imported_files_df[(imported_files_df["errors"] > 0) | (imported_files_df["import_errors"] > 0)][["name", "errors", "warnings", "import_status", "import_errors", "import_warnings", "path"]].sort_values("name")


,name,errors,warnings,import_status,import_errors,import_warnings,path


In [42]:
file_number=list(errors.index.unique())
if len(file_number) > 0:
    rpt = tlnb.get_translation_report(imported_files_df, file_number)
    print(rpt)
else:
    print("No errors found")



NameError: name 'errors' is not defined

### Get the import report for a file

In case there are errors in the import phase check the import report.

In [48]:
ierrors = imported_files_df[imported_files_df.import_errors>0][["import_status", "import_errors","import_warnings","name","imported","errors","warnings","path"]].sort_values("name")
ierrors

,import_status,import_errors,import_warnings,name,imported,errors,warnings,path


In [47]:

rpt = tlnb.get_import_rpt(imported_files_df, rows=list(ierrors.index), match_path=True)
print(rpt)

In [45]:
tlnb.get_import_rpt('sources/dehergne-a.cli')

'No import report found'